In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/")

In [2]:
train=pd.read_csv("numerai_training_data.csv")
test=pd.read_csv("numerai_tournament_data.csv")

In [ ]:
train.head()

In [ ]:
from sklearn.cross_validation import StratifiedKFold
skf=StratifiedKFold(train['target'],n_folds=5)
i=1
for train_index, test_index in skf:
    print "Train:",train_index,"Test:",test_index
    train.set_value(test_index,'group',i)
    i=i+1

In [ ]:
train.to_csv("numerai_training_data_withFolds.csv",index=False)

In [3]:
train_grouped=pd.read_csv("numerai_training_data_withFolds.csv")

In [5]:
# Custom Function created which would do the CV Manually
# Function should be created in such a way that proper calibration should be done w.r.t the LB.. with proper calibration
# submission could be done cautiously

target = 'target'
IDCol = ['t_id']

import random
random.seed(111)

from sklearn import cross_validation, metrics
from datetime import datetime

def custModelfit(alg2R,dtrain,dtest,predictors,target,IDCol,filename,postfix=True,verbose=True,stack=False):
    print datetime.now().time()
    Model_Score=[]
    CV_Score=[]
    predOutofFoldDF=[]
    
    for i in range(5):
        
        alg2R.fit(dtrain.loc[dtrain['group']%5!=i,predictors],dtrain.loc[dtrain['group']%5!=i,target])        
        dtrain_predprob_model =alg2R.predict_proba(dtrain.loc[dtrain['group']%5!=i,predictors])[:,1]
        idsForTrainInFold=dtrain.loc[dtrain['group']%5!=i,"t_id"]
        predInFoldDF = pd.DataFrame({"t_id":idsForTrainInFold,"predInFold":dtrain_predprob_model})
        
        dtrain_predprob=alg2R.predict_proba(dtrain.loc[dtrain['group']%5==i,predictors])[:,1]
        idsForTrainOutofFold =  dtrain.loc[dtrain['group']%5==i,"t_id"]
        predOutofFoldDF.append(pd.DataFrame({"t_id":idsForTrainOutofFold,"predOutofFold":dtrain_predprob}))
        
#         dtrain.loc[dtrain['group']%5!=i,"preds"]=alg2R.predict_proba(dtrain.loc[dtrain['group']%5!=i,predictors])[:,1]
#         dtrain.loc[dtrain['group']%5==i,"preds"]=alg2R.predict_proba(dtrain.loc[dtrain['group']%5==i,predictors])[:,1]
                
#         if postfix==True:
#             dtrain.loc[dtrain['var15']<=22,"preds"]=0

        if stack==True:
            dtrain.loc[dtrain['group']%5!=i,"pred" + str(i)]=alg2R.predict_proba(dtrain.loc[dtrain['group']%5!=i,predictors])[:,1]
            dtrain.loc[dtrain['group']%5==i,"pred" + str(i)]=alg2R.predict_proba(dtrain.loc[dtrain['group']%5==i,predictors])[:,1]
        
        #metrics.roc_auc_score(abc["TARGET"]).values,abc["preds"].values)
        train_predDF=dtrain[['t_id','target','group']]
        train_predDF = pd.merge(train_predDF,predInFoldDF,on=['t_id'])
        Model_Score.append(metrics.log_loss(train_predDF.loc[train_predDF['group']%5!=i,target].values,train_predDF.loc[train_predDF['group']%5!=i,'predInFold'].values))

        train_predDF=dtrain[['t_id','target','group']]
        train_predDF = pd.merge(train_predDF,predOutofFoldDF[i],on=['t_id'])
        CV_Score.append(metrics.log_loss(train_predDF.loc[train_predDF['group']%5==i,target].values,train_predDF.loc[train_predDF['group']%5==i,'predOutofFold'].values))
        
        if verbose == True:
            print "Log_Loss : %.4g" % (Model_Score[i])
            print "CV Log_Loss : %.4g" % (CV_Score[i])
        
    print "Overall Model Score : %.4g" %(np.mean(Model_Score))
    print "Overall CV Score %.4g" %(np.mean(CV_Score))
    
    #Predict on testing data
    alg2R.fit(dtrain[predictors],dtrain[target])
    dtest[target]=alg2R.predict_proba(dtest[predictors])[:,1]
    
    #Export CV file
    predOutofFoldDF_CV = pd.concat(predOutofFoldDF,ignore_index=True)
    #cv = pd.DataFrame(dtrain[['t_id','predOutofFold']])
    predOutofFoldDF_CV = predOutofFoldDF_CV.rename(columns={'predOutofFold': "CV_"+filename[:-4]})
    if not os.path.exists("CV"):
        os.makedirs("CV")
    predOutofFoldDF_CV.to_csv("CV/"+"CV_"+filename,index=False)
       
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission = submission.rename(columns={'target': 'probability'})
    if not os.path.exists("submission"):
        os.makedirs("submission")
    submission.to_csv("submission/"+filename,index=False)
    
    print datetime.now().time()
    
    if stack==True:
        dtrain['pred1'] = dtrain['pred1'].fillna(0.0)
        dtrain['pred2'] = dtrain['pred2'].fillna(0.0)
        dtrain['pred3'] = dtrain['pred3'].fillna(0.0)
        dtrain['pred4'] = dtrain['pred4'].fillna(0.0)
        dtrain['pred0'] = dtrain['pred0'].fillna(0.0)
        return((dtrain["pred1"]+dtrain["pred2"]+dtrain["pred3"]+dtrain["pred4"]+dtrain["pred0"])/5.0)
        # []"ID","PRED" whole train data set
        # retrun dataframe ...
   

In [36]:
# Log_Loss : 0.6911
# CV Log_Loss : 0.6915
# Log_Loss : 0.6912
# CV Log_Loss : 0.6911
# Log_Loss : 0.691
# CV Log_Loss : 0.6922
# Log_Loss : 0.691
# CV Log_Loss : 0.6922
# Log_Loss : 0.6912
# CV Log_Loss : 0.6911
# Overall Model Score : 0.6911
# Overall CV Score 0.6916
from sklearn.linear_model import LogisticRegression
alg3=LogisticRegression()
predictors=[x for x in train_grouped.columns if x not in [target]+IDCol+["predictions","group"]]
custModelfit(alg3,train_grouped,test,predictors,target,IDCol,'alg_Logistic.csv')


15:17:48.187000
Log_Loss : 0.6911
CV Log_Loss : 0.6915
Log_Loss : 0.6912
CV Log_Loss : 0.6911
Log_Loss : 0.691
CV Log_Loss : 0.6922
Log_Loss : 0.691
CV Log_Loss : 0.6922
Log_Loss : 0.6912
CV Log_Loss : 0.6911
Overall Model Score : 0.6911
Overall CV Score 0.6916
15:17:52.227000


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
alg3=LogisticRegression()
bag3=BaggingClassifier(base_estimator=alg3,n_estimators=20,n_jobs=-1)
predictors=[x for x in train_grouped.columns if x not in [target]+IDCol+["predictions","group"]]
custModelfit(bag3,train_grouped,test,predictors,target,IDCol,'alg_Logistic_Bag.csv')

20:40:14.198000
Log_Loss : 0.6911
CV Log_Loss : 0.6915
Log_Loss : 0.6912
CV Log_Loss : 0.6911
Log_Loss : 0.691
CV Log_Loss : 0.6924
Log_Loss : 0.691
CV Log_Loss : 0.6922
Log_Loss : 0.6912
CV Log_Loss : 0.6911
Overall Model Score : 0.6911
Overall CV Score 0.6917
20:41:57.066000


In [10]:
from sklearn.ensemble import RandomForestClassifier
alg1=RandomForestClassifier(n_estimators=200,max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',n_jobs=-1,random_state=111)
predictors=[x for x in train_grouped.columns if x not in [target]+IDCol+["predictions","group"]]
custModelfit(alg1,train_grouped,test,predictors,target,IDCol,'alg_RandomForest.csv')

20:39:34.253000
Log_Loss : 0.6871
CV Log_Loss : 0.6914
Log_Loss : 0.6874
CV Log_Loss : 0.6908
Log_Loss : 0.6872
CV Log_Loss : 0.6913
Log_Loss : 0.6869
CV Log_Loss : 0.6919
Log_Loss : 0.6873
CV Log_Loss : 0.6907
Overall Model Score : 0.6872
Overall CV Score 0.6912
20:40:35.854000


In [13]:
from sklearn.tree import DecisionTreeClassifier
alg2R=DecisionTreeClassifier(max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',random_state=111)
custModelfit(alg2R,train_grouped,test,predictors,target,IDCol,'alg_DTree.csv')

20:45:34.268000
Log_Loss : 0.689
CV Log_Loss : 0.6952
Log_Loss : 0.689
CV Log_Loss : 0.6949
Log_Loss : 0.6895
CV Log_Loss : 0.6931
Log_Loss : 0.689
CV Log_Loss : 0.6933
Log_Loss : 0.6897
CV Log_Loss : 0.6947
Overall Model Score : 0.6893
Overall CV Score 0.6942
20:45:35.772000


In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
alg3=DecisionTreeClassifier(max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',random_state=111)
ada1=AdaBoostClassifier(base_estimator=alg3,n_estimators=200)
custModelfit(ada1,train_grouped,test,predictors,target,IDCol,'alg_DTree_AdaBoost.csv')

20:47:59.022000
Log_Loss : 0.6905
CV Log_Loss : 0.6931
Log_Loss : 0.6906
CV Log_Loss : 0.6931
Log_Loss : 0.6906
CV Log_Loss : 0.6931
Log_Loss : 0.6905
CV Log_Loss : 0.6931
Log_Loss : 0.6907
CV Log_Loss : 0.693
Overall Model Score : 0.6906
Overall CV Score 0.6931
20:50:06.255000


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
alg3=LogisticRegression(solver='sag')
ada1=AdaBoostClassifier(base_estimator=alg3)
custModelfit(ada1,train_grouped,test,predictors,target,IDCol,'alg_Logistic_AdaB.csv')

20:51:28.945000
Log_Loss : 0.6931
CV Log_Loss : 0.6931
Log_Loss : 0.6931
CV Log_Loss : 0.6931
Log_Loss : 0.6931
CV Log_Loss : 0.6931
Log_Loss : 0.6931
CV Log_Loss : 0.6931
Log_Loss : 0.6931
CV Log_Loss : 0.6931
Overall Model Score : 0.6931
Overall CV Score 0.6931
21:02:08.399000


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
alg1=RandomForestClassifier(n_estimators=50,max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',n_jobs=-1,random_state=111)
ada1=AdaBoostClassifier(base_estimator=alg1,n_estimators=100)
predictors=[x for x in train_grouped.columns if x not in [target]+IDCol+["predictions","group"]]
custModelfit(alg1,train_grouped,test,predictors,target,IDCol,'alg_RandomForest_AdaB.csv')

21:03:51.025000
Log_Loss : 0.6873
CV Log_Loss : 0.6914
Log_Loss : 0.6875
CV Log_Loss : 0.6907
Log_Loss : 0.6873
CV Log_Loss : 0.6913
Log_Loss : 0.6869
CV Log_Loss : 0.692
Log_Loss : 0.6874
CV Log_Loss : 0.6907
Overall Model Score : 0.6873
Overall CV Score 0.6912
21:04:02.060000


In [21]:
from sklearn.tree import ExtraTreeClassifier
algET=ExtraTreeClassifier(max_depth=5,min_samples_split=50, min_samples_leaf=25,max_features='sqrt',random_state=111)
custModelfit(algET,train_grouped,test,predictors,target,IDCol,'alg_ET.csv')

21:26:54.610000
Log_Loss : 0.6912
CV Log_Loss : 0.6929
Log_Loss : 0.6913
CV Log_Loss : 0.6923
Log_Loss : 0.6913
CV Log_Loss : 0.6926
Log_Loss : 0.6913
CV Log_Loss : 0.6933
Log_Loss : 0.6913
CV Log_Loss : 0.6919
Overall Model Score : 0.6913
Overall CV Score 0.6926
21:26:55.676000


In [22]:
from sklearn.neighbors import KNeighborsClassifier
algKNN=KNeighborsClassifier(leaf_size=100,n_jobs=4,n_neighbors=20)
custModelfit(algKNN,train_grouped,test,predictors,target,IDCol,'alg_KNN.csv')

21:27:29.588000
Log_Loss : 0.6629
CV Log_Loss : 0.7146
Log_Loss : 0.6627
CV Log_Loss : 0.7136
Log_Loss : 0.6634
CV Log_Loss : 0.7189
Log_Loss : 0.6623
CV Log_Loss : 0.717
Log_Loss : 0.663
CV Log_Loss : 0.7124
Overall Model Score : 0.6629
Overall CV Score 0.7153
21:34:22.407000


In [26]:
from sklearn.naive_bayes import GaussianNB
algGNB=GaussianNB()
custModelfit(algGNB,train_grouped,test,predictors,target,IDCol,'alg_GNB.csv')

21:37:01.976000
Log_Loss : 0.7007
CV Log_Loss : 0.702
Log_Loss : 0.7002
CV Log_Loss : 0.6993
Log_Loss : 0.701
CV Log_Loss : 0.702
Log_Loss : 0.7023
CV Log_Loss : 0.7055
Log_Loss : 0.7009
CV Log_Loss : 0.6998
Overall Model Score : 0.701
Overall CV Score 0.7017
21:37:03.257000


In [25]:
from sklearn.svm import SVC
alg8=SVC(random_state=111)
custModelfit(alg1,train_grouped,test,predictors,target,IDCol,'alg_svm.csv')

21:35:33.292000
Log_Loss : 0.6873
CV Log_Loss : 0.6914
Log_Loss : 0.6875
CV Log_Loss : 0.6907
Log_Loss : 0.6873
CV Log_Loss : 0.6913
Log_Loss : 0.6869
CV Log_Loss : 0.692
Log_Loss : 0.6874
CV Log_Loss : 0.6907
Overall Model Score : 0.6873
Overall CV Score 0.6912
21:35:44.621000


In [28]:
from sklearn.linear_model import SGDClassifier
alg8=SGDClassifier(loss='log')
custModelfit(alg8,train_grouped,test,predictors,target,IDCol,'alg_SGD.csv')

21:37:25.508000
Log_Loss : 0.7284
CV Log_Loss : 0.7306
Log_Loss : 0.6984
CV Log_Loss : 0.6991
Log_Loss : 0.7168
CV Log_Loss : 0.7175
Log_Loss : 0.7458
CV Log_Loss : 0.7484
Log_Loss : 0.7087
CV Log_Loss : 0.7073
Overall Model Score : 0.7196
Overall CV Score 0.7206
21:37:27.097000


In [29]:
from sklearn.ensemble import GradientBoostingClassifier
alg1=GradientBoostingClassifier(random_state=111)
custModelfit(alg1,train_grouped,test,predictors,target,IDCol,'alg_GBM.csv')

21:38:18.060000
Log_Loss : 0.6811
CV Log_Loss : 0.6919
Log_Loss : 0.6812
CV Log_Loss : 0.6916
Log_Loss : 0.6813
CV Log_Loss : 0.6921
Log_Loss : 0.6809
CV Log_Loss : 0.6928
Log_Loss : 0.6813
CV Log_Loss : 0.6914
Overall Model Score : 0.6811
Overall CV Score 0.692
21:40:04.350000


In [6]:
# Function for xgboost
import xgboost as xgb
from datetime import datetime
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
predictors=[x for x in train.columns if x not in [target]+IDCol+["predictions"]]
def xGBmodelfit(alg,dtrain,dtest,predictors,filename,useTrainCV=True, cv_folds=5,early_stopping_rounds=200):
    print datetime.now().time()
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values,label = dtrain[target].values)
        cvresult = xgb.cv(xgb_param,xgtrain,num_boost_round=alg.get_params()['n_estimators'],
            nfold=cv_folds,metrics='logloss',early_stopping_rounds=early_stopping_rounds)#,show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])#,booster='gbtree')
    
    bestNoOfRounds = cvresult['test-logloss-mean'].argmax()
    bestAnsForThisSetting =  cvresult.ix[bestNoOfRounds]
    bestModelEval = bestAnsForThisSetting[0]
    print "Best Num of Rounds %.7g" %bestNoOfRounds
#     print "Best Ans %.7g" %bestAnsForThisSetting
    print "Best Model Eval (Test-LogLoss) %.7g" %bestModelEval
    
    alg.fit(dtrain[predictors],dtrain[target],eval_metric='logloss')#,eta=0.0001)
    
#   Predicting training data
    dtrain["predictions"] = alg.predict_proba(dtrain[predictors])[:,1]
    dtrain_predprob=alg.predict_proba(dtrain[predictors])[:,1]       
    
    print "\nModel Report"
    print "Accuracy : %.4g" %metrics.accuracy_score(dtrain[target],alg.predict(dtrain[predictors]))
    print "LogLoss Score (train) : %f" %metrics.log_loss(dtrain[target],dtrain_predprob)
    
#   Feature important plot
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar',title = 'Feature Importance')
    plt.ylabel('Feature Importance score')
    
    #Predict on testing data
    dtest[target]=alg.predict_proba(dtest[predictors])[:,1]
    
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission = submission.rename(columns={'target': 'probability'})
    
    if not os.path.exists("submission"):
        os.makedirs("submission")
    submission.to_csv("submission/"+filename,index=False)
    
    cv = pd.DataFrame(dtrain[['t_id','predictions']])
    cv = cv.rename(columns={'predictions': "CV_"+filename[:-4]})
    if not os.path.exists("CV"):
        os.makedirs("CV")
    cv.to_csv("CV/"+"CV_"+filename,index=False)
    
    print datetime.now().time()

In [14]:
# CV = 0.6929616/0.666240, LB = 0.69468

xgb1 = XGBClassifier(
objective = 'binary:logistic',
learning_rate=0.1,
n_estimators = 100,
max_depth=3,
min_child_weight=1,
subsample=0.8,
colsample_bytree=0.8,
nthread=4,
seed=27)

xGBmodelfit(xgb1,train,test,predictors,'xgBoost.csv')

15:56:15.026000
Best Num of Rounds 0
Best Model Eval (Test-LogLoss) 0.6928388

Model Report
Accuracy : 0.5636
LogLoss Score (train) : 0.683068
15:56:29.952000


In [13]:
from sklearn.grid_search import GridSearchCV
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100,
 gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.69186, std: 0.00061, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: -0.69194, std: 0.00057, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: -0.69196, std: 0.00061, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: -0.69376, std: 0.00116, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: -0.69350, std: 0.00116, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: -0.69365, std: 0.00100, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: -0.69736, std: 0.00132, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: -0.69619, std: 0.00145, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: -0.69614, std: 0.00105, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: -0.70421, std: 0.00271, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: -0.70197, std: 0.00178, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: -0.70113, std: 0.00178, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_chil